### What would happen if you ran all the common models and compared them?



In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn import linear_model
from sklearn.metrics import mean_squared_error, r2_score

import math 

This is my scoring function, row append sucks but it works, would not reccomend this for large dataframes 

In [ ]:
model_metrics = pd.DataFrame(columns=["Model", "MSE", "RMSE", "R2"])


def metrics(model, y, y_hat):

    model_metrics.loc[-1] = {"Model" : model, 
              "MSE" : mean_squared_error(y, y_hat),
              "RMSE" : math.sqrt(mean_squared_error(y, y_hat)),
              "R2" : r2_score(y, y_hat)}

    model_metrics.index = model_metrics.index + 1
    return model_metrics 

#metrics("PCA Forest",y_test, epa_pca_y_pred)

Import some data, this is the 2018 EPA Auto dataset about 4000 rows have already been dropped that were useles, you can go backwards in my blog to see what i did and why. I orignally did the data engineering in R, but whatever. https://sqlshep.com/?p=815



In [ ]:
epa = pd.read_csv('https://raw.githubusercontent.com/sqlshep/SQLShepBlog/master/data/epaMpg.csv')

In [ ]:
epa.head(10)

In [ ]:
epa.shape

In [ ]:
epa.describe().T


Engineer the dataset a bit, some models dislike periods in the name, drop the first 3 columns, i do not want th emodel to fit to manufacturer and some of the columns are descritptions of other columns so they are dupes. 

In [ ]:
#Drop the row number
epa = epa.drop(epa.columns[[0]], axis=1)

#replace the "." in the column names with "_"
epa.columns = epa.columns.str.replace('.', '_')

# Drop the first three columns
epa = epa.drop(epa.columns[[0,1,2]], axis=1)

# drop descrition columns
epa = epa.drop(epa.columns[[3,9,11]], axis=1)
epa



Visualize some stuff, see if there are any patterns. 

In [ ]:
epa.plot.scatter(x='HorsePower',
                      y='Cylinders',
                      c='DarkBlue',
                figsize=(15,5))

In [ ]:
epa.plot.scatter(x='HorsePower',
                      y='Weight',
                      c='DarkBlue',
                figsize=(15,5))

In [ ]:
epa.plot.scatter(x='HorsePower',
                      y='FuelEcon',
                      c='DarkBlue',
                figsize=(15,5))

In [ ]:
plot = pd.plotting.scatter_matrix(epa,
                figsize=(15,15))



In [ ]:
epa.corr()

Any correlations? 

In [ ]:
sns.set(rc={'figure.figsize':(12,8)})
sns.heatmap(epa.corr(), annot = True)



In [ ]:
epa['Tested_Transmission_Type_Code']= epa['Tested_Transmission_Type_Code'].astype('category')
    
epa['Drive_System_Code']= epa['Drive_System_Code'].astype('category')

In [ ]:
epa.shape

In [ ]:
#One hot encode categories
epa = pd.get_dummies(epa)

In [ ]:
print(epa.shape)
epa

In [ ]:
# Create the training dataset for scikit learn, you will need all varialbes except the labe you are trying to predict
epa_X = epa.iloc[:, epa.columns !='FuelEcon']
#epa_X = epa.iloc[:, epa.columns =='Weight']
epa_X

In [ ]:
epa_y = epa.iloc[:, epa.columns =='FuelEcon']

In [ ]:
# Split the training and test set 
X_train, X_test, y_train, y_test = train_test_split(epa_X, epa_y, test_size=0.20)

In [ ]:
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape )

In [ ]:
from sklearn import linear_model

epa_lm = regr = linear_model.LinearRegression()

In [ ]:
epa_lm.fit(X_train, y_train)

In [ ]:
epa_y_pred = epa_lm.predict(X_test)

In [ ]:
metrics("linear_model",y_test, epa_y_pred)

In [ ]:
from sklearn.model_selection import cross_val_predict

# cross_val_predict returns an array of the same size as `y` where each entry
# is a prediction obtained by cross validation:
predicted = cross_val_predict(epa_lm, X_test, y_test, cv=5)

fig, ax = plt.subplots()
ax.scatter(y_test, predicted, edgecolors=(0, 0, 0))
ax.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'k--', lw=4)
ax.set_xlabel('Measured')
ax.set_ylabel('Predicted')
plt.show()



# K Nearest Neighbor Regressor


In [ ]:
# Create and fit a nearest-neighbor classifier
from sklearn.neighbors import KNeighborsRegressor


In [ ]:
knn = KNeighborsRegressor()
knn.fit(X_train, y_train)
knn_y_hat = knn.predict(X_test)

In [ ]:
metrics("KNeighborsRegressor",y_test, knn_y_hat)

# Random Forest Regressor

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn import tree
import graphviz 

In [ ]:
epa_forest = RandomForestRegressor()
epa_forest.fit(X_train, y_train)

In [ ]:
forest_y_hat = epa_forest.predict(X_test)

In [ ]:
metrics("RandomForestRegressor",y_test, forest_y_hat)

# Gradient Boosting for regression

In [ ]:
from sklearn.ensemble import GradientBoostingRegressor

In [ ]:
reg = GradientBoostingRegressor(random_state=0)

In [ ]:
reg.fit(X_train, y_train)

In [ ]:
gb_y_hat = reg.predict(X_test)


In [ ]:
metrics("GradientBoostingRegressor",y_test, gb_y_hat)

# Principal component analysis (PCA)

Ever wondered what PCA would do to your prediction?  

In [ ]:
from sklearn.decomposition import PCA

In [ ]:
pca = PCA(n_components=8)
principalComponents = pca.fit_transform(epa_X)
principalDf = pd.DataFrame(data = principalComponents, columns = ['V1','V2','V3','V4','V5','V6','V7','V8'])

In [ ]:
principalDf

In [ ]:
y = pd.DataFrame(epa["FuelEcon"] )
pd.concat([principalDf,y],axis=1)

sns.set(rc={'figure.figsize':(12,8)})
sns.heatmap(pd.concat([principalDf,y],axis=1).corr(), annot = True)

In [ ]:
epa_forest = RandomForestRegressor()
epa_forest.fit(X_train, y_train)
epa_pca_y_pred = epa_forest.predict(X_test)

In [ ]:
metrics("PCA W/ RandomForestRegressor",y_test, epa_pca_y_pred)